In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import geograpy as gg
import pycountry as pc

In [47]:
df = pd.read_csv("output_combined.csv")
df.head()

,id,username,text,retweets,favorites,replies,permalink,date,mentions,hashtags
0,1200516560160481280,vanderfullife,Pan Pacific Airlines adds Clark – Muan route f...,0,0,0,https://twitter.com/vanderfullife/status/12005...,2019-11-29 20:47:07+00:00,NaN,#news #today #travel #vacation #cruise #holida...
1,1200516487389122560,sweetsteph0914,Today I was presented w/ an #eviction notice (...,0,0,0,https://twitter.com/sweetsteph0914/status/1200...,2019-11-29 20:46:49+00:00,@Twitter,#eviction #holidays
2,1200516453612568576,PhilMorseRE,Earning extra money during the #holidays can m...,0,0,0,https://twitter.com/PhilMorseRE/status/1200516...,2019-11-29 20:46:41+00:00,NaN,#holidays #creditcards
3,1200516306967154688,HollyNorth,"Oh the weather outside is frightful, and our s...",0,0,0,https://twitter.com/HollyNorth/status/12005163...,2019-11-29 20:46:06+00:00,NaN,#Holidays #HollyNorth #HollyNorthProductionSup...
4,1200516102205239296,LissaandTiniRay,Hanging with some of my favs @CranbrookLiquo1 ...,0,0,0,https://twitter.com/LissaandTiniRay/status/120...,2019-11-29 20:45:17+00:00,@CranbrookLiquo1,#g4tequila #liquorstores #tastings #lovethiscr...


In [55]:
tknzr = TweetTokenizer()
stop_words = set(stopwords.words('english'))

#Use these for output csv that have location
# df['location_filled'] = df['location'].fillna("")
# df['Location_Token'] = df['location_filled'].apply(lambda x: x.split(","))
# df['Tweet_Token'] = df['tweet'].apply(lambda x: tknzr.tokenize(x))
df['Tweet_Token'] = df['text'].apply(lambda x: tknzr.tokenize(x))
df['Tweet_Token_RemoveStop'] = df['Tweet_Token'].apply(lambda x: [w for w in x if not w in stop_words])

In [49]:
#List of Location from pycountry.country & subdivision
location_list = []
for country in list(pc.countries):
    print(country)
    location_list.append(country.alpha_2.lower())
    location_list.append(country.alpha_3.lower())
    location_list.append(country.name.lower())
    try:
        location_list.append(country.common_name.lower())
    except:
        None
for city in list(pc.subdivisions):
    location_list.append(city.name.lower())
    print(city)
    try:
        location_list.append(city.city.code[-2:].lower())
    except:
        None
location_list.append('philadelphia')
location_list.append('fl')
location_list.append('nyc')
Remove_list = ['and', 'as', 'at', 'can', 'in', 'is','me', 'my','it','us','to','by','most','be','so']
for remove_word in Remove_list:
    location_list.remove(remove_word)

Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533')
Country(alpha_2='AF', alpha_3='AFG', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan')
Country(alpha_2='AO', alpha_3='AGO', name='Angola', numeric='024', official_name='Republic of Angola')
Country(alpha_2='AI', alpha_3='AIA', name='Anguilla', numeric='660')
Country(alpha_2='AX', alpha_3='ALA', name='Åland Islands', numeric='248')
Country(alpha_2='AL', alpha_3='ALB', name='Albania', numeric='008', official_name='Republic of Albania')
Country(alpha_2='AD', alpha_3='AND', name='Andorra', numeric='020', official_name='Principality of Andorra')
Country(alpha_2='AE', alpha_3='ARE', name='United Arab Emirates', numeric='784')
Country(alpha_2='AR', alpha_3='ARG', name='Argentina', numeric='032', official_name='Argentine Republic')
Country(alpha_2='AM', alpha_3='ARM', name='Armenia', numeric='051', official_name='Republic of Armenia')
Country(alpha_2='AS', alpha_3='ASM', name='American Samoa', n

Subdivision(code='GR-84', country_code='GR', name='Samos', parent='K', parent_code='GR-K', type='Department')
Subdivision(code='GR-85', country_code='GR', name='Chios', parent='K', parent_code='GR-K', type='Department')
Subdivision(code='GR-91', country_code='GR', name='Irakleio', parent='M', parent_code='GR-M', type='Department')
Subdivision(code='GR-92', country_code='GR', name='Lasithi', parent='M', parent_code='GR-M', type='Department')
Subdivision(code='GR-93', country_code='GR', name='Rethymno', parent='M', parent_code='GR-M', type='Department')
Subdivision(code='GR-94', country_code='GR', name='Chania', parent='M', parent_code='GR-M', type='Department')
Subdivision(code='GR-A', country_code='GR', name='Anatoliki Makedonia kai Thraki', parent_code=None, type='Administrative region')
Subdivision(code='GR-A1', country_code='GR', name='Attiki', parent='I', parent_code='GR-I', type='Department')
Subdivision(code='GR-B', country_code='GR', name='Kentriki Makedonia', parent_code=None, 

Subdivision(code='MK-63', country_code='MK', name='Probištip', parent_code=None, type='Municipality')
Subdivision(code='MK-64', country_code='MK', name='Radoviš', parent_code=None, type='Municipality')
Subdivision(code='MK-65', country_code='MK', name='Rankovce', parent_code=None, type='Municipality')
Subdivision(code='MK-66', country_code='MK', name='Resen', parent_code=None, type='Municipality')
Subdivision(code='MK-67', country_code='MK', name='Rosoman', parent_code=None, type='Municipality')
Subdivision(code='MK-68', country_code='MK', name='Saraj', parent_code=None, type='Municipality')
Subdivision(code='MK-69', country_code='MK', name='Sveti Nikole', parent_code=None, type='Municipality')
Subdivision(code='MK-70', country_code='MK', name='Sopište', parent_code=None, type='Municipality')
Subdivision(code='MK-71', country_code='MK', name='Staro Nagoričane', parent_code=None, type='Municipality')
Subdivision(code='MK-72', country_code='MK', name='Struga', parent_code=None, type='Mun

Subdivision(code='TH-56', country_code='TH', name='Phayao', parent_code=None, type='Province')
Subdivision(code='TH-57', country_code='TH', name='Chiang Rai', parent_code=None, type='Province')
Subdivision(code='TH-58', country_code='TH', name='Mae Hong Son', parent_code=None, type='Province')
Subdivision(code='TH-60', country_code='TH', name='Nakhon Sawan', parent_code=None, type='Province')
Subdivision(code='TH-61', country_code='TH', name='Uthai Thani', parent_code=None, type='Province')
Subdivision(code='TH-62', country_code='TH', name='Kamphaeng Phet', parent_code=None, type='Province')
Subdivision(code='TH-63', country_code='TH', name='Tak', parent_code=None, type='Province')
Subdivision(code='TH-64', country_code='TH', name='Sukhothai', parent_code=None, type='Province')
Subdivision(code='TH-65', country_code='TH', name='Phitsanulok', parent_code=None, type='Province')
Subdivision(code='TH-66', country_code='TH', name='Phichit', parent_code=None, type='Province')
Subdivision(cod

In [50]:
def country_name(lists):
    if lists == []:
        return np.nan
    else:
        lower_list = []
        Location = []
        for token in lists:
            lower_list.append(token.lower())
        for location in location_list:
            if location in lower_list:
                Location.append(location)
        if Location == []:
            for value in lower_list:
                fuzz_score_country = {}
                for country in location_list:
                    fuzz_score_country[country] = fuzz.ratio(value,country)
                for country in ['united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
                    fuzz_score_country[country] = fuzz.ratio(value,country)
                for country in ['the netherlands','england','u.k.','uk','uk ',' uk','uae','korea','south korea']:
                    fuzz_score_country[country] = fuzz.ratio(value,country)
            if max(fuzz_score_country, key=fuzz_score_country.get) in ['cascadia','united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
                Location.append(pc.countries.lookup('united states').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['england','u.k.','uk','uk ',' uk']:
                Location.append(pc.countries.lookup('united kingdom').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['uae']:
                Location.append(pc.countries.lookup('United Arab Emirates').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['korea','south korea']:
                Location.append(pc.countries.lookup('KR').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['the netherlands']:
                Location.append(pc.countries.lookup('netherlands').alpha_3)
            elif fuzz_score_country[max(fuzz_score_country, key=fuzz_score_country.get)] <70:
                if Location == []:
                    return np.nan
                else:
                    return Location
            else:
                Location.append(max(fuzz_score_country, key=fuzz_score_country.get))
                return Location
        else:
            return Location
                
#                 fuzz_score_city = {}
#                 for city in city_list:
#                     fuzz_score_city[city] = fuzz.ratio(value,city)
#                 if fuzz_score_city[max(fuzz_score_city, key=fuzz_score_city.get)] <90:
#                     return np.nan
#                 else:
#                     return pc.countries.lookup(pc.subdivisions.lookup(max(fuzz_score_city, key=fuzz_score_city.get)).country_code).alpha_3
#             return pc.countries.lookup(max(fuzz_score_country, key=fuzz_score_country.get)).alpha_3
#         print(fuzz_score_country[max(fuzz_score_country)])
        


In [51]:
#An example of a good tweet
# country_name(df['Tweet_Token'][984])

In [52]:
#Apply country_name on locations
# df['Location_Found'] = df['Location_Token'].apply(lambda x: country_name(x))
# df['Location_Found']

In [53]:
#Apply country_name on tweets
df['Tweet_Location_Found'] = df['Tweet_Token_RemoveStop'].apply(lambda x: country_name(x))
df['Tweet_Location_Found']

KeyboardInterrupt: 

In [ ]:
df.to_csv('output_combined_location.csv',index=False)
pd.read_csv("output_combined_location.csv")

In [45]:
#Run this to get all valid locations, remove empty location records
df_copy = df.copy()
df_copy = df_copy.dropna(subset = ['Tweet_Location_Found'])
print(df_copy['Tweet_Location_Found'])

5                  [switzerland]
7                  [philippines]
9                           [de]
10                          [rs]
14                  [de, et, la]
15                [south, south]
16                  [de, et, la]
18                         [cod]
25       [indonesia, pool, bali]
26                      [bhutan]
36              [southern, gulf]
39                          [so]
44                 [switzerland]
45                    [thailand]
47                [aruba, aruba]
52                 [uttarakhand]
55                     [morelos]
56                          [ms]
57                [south, south]
61                   [islamabad]
64                      [de, la]
66                        [york]
67                          [dm]
68                    [maldives]
73                      [canada]
74                          [la]
76                      [greece]
81                   [singapore]
83                        [bali]
84                [south, south]
          

In [63]:
#Run this to get all valid locations, remove empty location records
df_copy = pd.read_csv("output_combined_location.csv")
df_copy = df_copy.dropna(subset = ['Tweet_Location_Found'])
def stopwordfilter(lists):
    if lists is np.nan:
        print('empty')
        print(lists)
        return np.nan
    if len(lists) == 1:
        print('one element')
        print(lists)
        return lists
    else:
        lists = list(lists)
        print(type(lists))
        print(len(lists))
        print(lists)
        return [w for w in lists if not w in stop_words]
df_copy['Tweet_Location_Found'].apply(lambda x: stopwordfilter(x))
# df_copy['Tweet_Location_Found_Removestop'] = df_copy['Tweet_Location_Found'].apply(lambda x: [w for w in x if not w in stop_words])
# print(df_copy['Tweet_Location_Found_Removestop'])

<class 'list'>
7
['[', "'", 'p', 'a', 'n', "'", ']']
<class 'list'>
31
['[', "'", 'a', 't', "'", ',', ' ', "'", 'c', 'a', 'n', "'", ',', ' ', "'", 'm', 'e', "'", ',', ' ', "'", 'm', 'y', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
19
['[', "'", 'c', 'a', 'n', "'", ',', ' ', "'", 'i', 't', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
25
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 'i', 't', "'", ',', ' ', "'", 's', 'o', "'", ']']
<class 'list'>
13
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'm', 'y', "'", ']']
<class 'list'>
6
['[', "'", 'i', 'n', "'", ']']
<class 'list'>
6
['[', "'", 'a', 't', "'", ']']
<class 'list'>
7
['[', "'", 'a', 'r', 'e', "'", ']']
<class 'list'>
12
['[', "'", 'b', 'y', "'", ',', ' ', "'", 'i', 's', "'", ']']
<class 'list'>
18
['[', "'", 'a', 't', "'", ',', ' ', "'", 'b', 'e', "'", ',', ' ', "'", 'i', 'n', "'", ']']
<class 'list'>
13
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 's', 'o', "'", ']']
<class 'li

20
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'c', 'a', 'n', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
12
['[', "'", 'a', 't', "'", ',', ' ', "'", 'i', 't', "'", ']']
<class 'list'>
31
['[', "'", 'i', 's', "'", ',', ' ', "'", 'i', 't', "'", ',', ' ', "'", 's', 'o', "'", ',', ' ', "'", 't', 'o', "'", ',', ' ', "'", 'h', 'a', 's', "'", ']']
<class 'list'>
12
['[', "'", 'i', 'n', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
6
['[', "'", 's', 't', "'", ']']
<class 'list'>
19
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'p', 's', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
43
['[', "'", 'a', 'c', 'r', 'e', "'", ',', ' ', "'", 'l', 'o', 't', "'", ',', ' ', "'", 'f', 'l', 'o', 'r', 'i', 'd', 'a', "'", ',', ' ', "'", 'f', 'l', 'o', 'r', 'i', 'd', 'a', "'", ',', ' ', "'", 'f', 'l', "'", ']']
<class 'list'>
26
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'd', 'o', "'", ',', ' ', "'", 'i', 't', "'", ',', ' ', "'", 'h', 'a', 's', "'", ']']
<class 'list'>
7
['

<class 'list'>
6
['[', "'", 's', 'o', "'", ']']
<class 'list'>
13
['[', "'", 'i', 's', "'", ',', ' ', "'", 'h', 'a', 's', "'", ']']
<class 'list'>
20
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 'b', 'e', "'", ']']
<class 'list'>
20
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 'b', 'e', "'", ']']
<class 'list'>
13
['[', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
6
['[', "'", 'i', 't', "'", ']']
<class 'list'>
27
['[', "'", 'a', 't', "'", ',', ' ', "'", 'c', 'a', 'n', "'", ',', ' ', "'", 't', 'o', "'", ',', ' ', "'", 'm', 'o', 's', 't', "'", ']']
<class 'list'>
6
['[', "'", 'a', 't', "'", ']']
<class 'list'>
19
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 'm', 'y', "'", ']']
<class 'list'>
25
['[', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 'b', 'e', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
20
['[', "'", 'a', 'n',

<class 'list'>
20
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
6
['[', "'", 'n', 'o', "'", ']']
<class 'list'>
26
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 'd', 'o', "'", ',', ' ', "'", 'i', 's', "'", ']']
<class 'list'>
26
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 'd', 'o', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
26
['[', "'", 'd', 'e', "'", ',', ' ', "'", 'l', 'a', "'", ',', ' ', "'", 'v', 'e', 'n', "'", ',', ' ', "'", 'b', 'a', 'r', "'", ']']
<class 'list'>
37
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'b', 'y', "'", ',', ' ', "'", 'd', 'o', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 't', 'o', "'", ',', ' ', "'", 'u', 's', "'", ']']
<class 'list'>
9
['[', "'", 't', 'e', 'x', 'a', 's', "'", ']']
<class 'list'>
18
['[', "'", 'd', 'o', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'li

['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 'r', 'e', "'", ']']
<class 'list'>
31
['[', "'", 'f', 'r', 'a', 'n', 'c', 'e', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 'u', 's', "'", ',', ' ', "'", 'p', 'a', 'r', 'i', 's', "'", ']']
<class 'list'>
12
['[', "'", 'b', 'y', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
12
['[', "'", 't', 'o', "'", ',', ' ', "'", 'u', 's', "'", ']']
<class 'list'>
12
['[', "'", 'c', 'a', 'n', 'a', 'r', 'i', 'a', 's', "'", ']']
<class 'list'>
18
['[', "'", 'm', 'a', 'l', 'd', 'i', 'v', 'e', 's', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
12
['[', "'", 'd', 'e', "'", ',', ' ', "'", 'l', 'a', "'", ']']
<class 'list'>
18
['[', "'", 'i', 'n', "'", ',', ' ', "'", 'b', 'u', 'd', 'a', 'p', 'e', 's', 't', "'", ']']
<class 'list'>
12
['[', "'", 't', 'a', 'o', 'u', 'r', 'i', 'r', 't', "'", ']']
<class 'list'>
6
['[', "'", 'a', 't', "'", ']']
<class 'list'>
13
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'b', 'e', "'", ']']
<class 'li

...

['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 't', "'", ']']
<class 'list'>
19
['[', "'", 'i', 'n', "'", ',', ' ', "'", 'g', 'r', 'e', 'e', 'n', 'w', 'i', 'c', 'h', "'", ']']
<class 'list'>
12
['[', "'", 'i', 's', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
6
['[', "'", 'b', 'e', "'", ']']
<class 'list'>
31
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 's', "'", ',', ' ', "'", 'a', 't', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 'u', 's', "'", ']']
<class 'list'>
37
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 's', "'", ',', ' ', "'", 'b', 'y', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 'i', 't', "'", ']']
<class 'list'>
17
['[', "'", 'j', 'a', 'm', 'a', 'i', 'c', 'a', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
19
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'm', 'e', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
12
['[', "'", 'm', 'y', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
6
[

13
['[', "'", 'a', 'r', 'e', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
28
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'm', 'a', 'l', 't', 'a', "'", ',', ' ', "'", 's', 'o', 'u', 't', 'h', 'e', 'r', 'n', "'", ']']
<class 'list'>
25
['[', "'", 'a', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a', "'", ',', ' ', "'", 'v', 'i', 'c', 't', 'o', 'r', 'i', 'a', "'", ']']
<class 'list'>
6
['[', "'", 't', 'o', "'", ']']
<class 'list'>
9
['[', "'", 'i', 'n', 'd', 'i', 'a', "'", ']']
<class 'list'>
14
['[', "'", 'i', 'n', "'", ',', ' ', "'", 'm', 'o', 's', 't', "'", ']']
<class 'list'>
31
['[', "'", 'a', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a', "'", ',', ' ', "'", 's', 'o', 'u', 't', 'h', "'", ',', ' ', "'", 's', 'o', 'u', 't', 'h', "'", ']']
<class 'list'>
21
['[', "'", 'i', 'n', 'd', 'o', 'n', 'e', 's', 'i', 'a', "'", ',', ' ', "'", 'j', 'a', 'w', 'a', "'", ']']
<class 'list'>
21
['[', "'", 'i', 'n', 'd', 'o', 'n', 'e', 's', 'i', 'a', "'", ',', ' ', "'", 'j', 'a', 'w', 'a', "'", ']']
<class

['[', "'", 'a', 't', "'", ']']
<class 'list'>
23
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'a', 's', "'", ',', ' ', "'", 'f', 'r', 'a', 'n', 'c', 'e', "'", ']']
<class 'list'>
6
['[', "'", 'd', 'e', "'", ']']
<class 'list'>
24
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'f', 'i', 'n', 'l', 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 'n', "'", ']']
<class 'list'>
21
['[', "'", 'u', 's', 'a', "'", ',', ' ', "'", 'c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a', "'", ']']
<class 'list'>
6
['[', "'", 'n', 'o', "'", ']']
<class 'list'>
19
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
7
['[', "'", 'a', 'n', 'd', "'", ']']
<class 'list'>
33
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'b', 'e', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 'm', 'o', 's', 't', "'", ']']
<class 'list'>
6
['[', "'", 't', 'o', "'", ']']
<class 'list'>
6
['[', "'", 'u', 's', "'", ']']
<class 'list'>
19
['[', "'", 

['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 'n', "'", ']']
<class 'list'>
25
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'b', 'y', "'", ',', ' ', "'", 'i', 't', "'", ',', ' ', "'", 'r', 'e', "'", ']']
<class 'list'>
7
['[', "'", 'a', 'n', 'd', "'", ']']
<class 'list'>
6
['[', "'", 'i', 's', "'", ']']
<class 'list'>
19
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 'n', "'", ',', ' ', "'", 't', 'o', "'", ']']
<class 'list'>
6
['[', "'", 'i', 's', "'", ']']
<class 'list'>
12
['[', "'", 'a', 't', "'", ',', ' ', "'", 'i', 's', "'", ']']
<class 'list'>
12
['[', "'", 'd', 'o', "'", ',', ' ', "'", 'i', 'n', "'", ']']
<class 'list'>
12
['[', "'", 'i', 'n', "'", ',', ' ', "'", 'i', 's', "'", ']']
<class 'list'>
19
['[', "'", 'a', 'n', 'd', "'", ',', ' ', "'", 'i', 's', "'", ',', ' ', "'", 'i', 't', "'", ']']
<class 'list'>
6
['[', "'", 'i', 't', "'", ']']
<class 'list'>
6
['[', "'", 'a', 't', "'", ']']
<class 'list'>
6
['[', "'", 'i', 'n', "'", ']']
<class 'list'>
23
['[', "'", 'i'

0                                       [[, ', p, n, ', ]]
1        [[, ', ', ,,  , ', c, n, ', ,,  , ', e, ', ,, ...
2               [[, ', c, n, ', ,,  , ', ', ,,  , ', ', ]]
3        [[, ', n, ', ,,  , ', ', ,,  , ', ', ,,  , ', ...
4                              [[, ', n, ', ,,  , ', ', ]]
5                                          [[, ', n, ', ]]
6                                             [[, ', ', ]]
7                                       [[, ', r, e, ', ]]
8                              [[, ', b, ', ,,  , ', ', ]]
10           [[, ', ', ,,  , ', b, e, ', ,,  , ', n, ', ]]
11                             [[, ', n, ', ,,  , ', ', ]]
12                             [[, ', n, ', ,,  , ', ', ]]
13                    [[, ', c, h, e, ', ,,  , ', l, ', ]]
14       [[, ', n, ', ,,  , ', ', ,,  , ', u, r, k, e, ...
15                             [[, ', n, ', ,,  , ', ', ]]
16                             [[, ', n, ', ,,  , ', ', ]]
17                                            [[, ', ', 

In [195]:
#Testing
df['Location_Token'][980:990]

980                 [Alto,  TN]
981        [Thunder Bay/Ottawa]
982                   [Cardiff]
983                       [BCK]
984             [Oslo,  Norway]
985                          []
986                          []
987    [Altrincham,  South Mcr]
988                          []
989           [Cincinnati,  OH]
Name: Location_Token, dtype: object